## **<u>CSCI 381: PHW#3S - Individual (Arnan Khan)</u>**

#### Chapter 6

Named Database: AdventureWorks

\[ChatGPT \*new\*  Generates Propositions\]( https://chatgpt.com/share/670fcc9e-bd20-800f-8a01-0e1dd896f7de )

# AdventureWorks Set Operators Questions

## 1\. Compare Person and Employees:

Find out which Persons are not listed as Employee by comparing the `Person` table and <span style="font-size: 12px;">Employee&nbsp;</span>  table.

In [10]:
WITH Non_Employees AS (
(
    SELECT BusinessEntityID
    FROM Person.Person
) 
EXCEPT
(
    SELECT BusinessEntityID
    FROM HumanResources.Employee
) 
)

SELECT TOP(5) FirstName, LastName
FROM Non_Employees as NE
INNER JOIN Person.Person as P on P.BusinessEntityID = NE.BusinessEntityID

(5 rows affected)

Total execution time: 00:00:00.030

FirstName,LastName
Gustavo,Achong
Catherine,Abel
Kim,Abercrombie
Humberto,Acevedo
Pilar,Ackerman


## 2. Compare Sales Territories:
Determine the regions where AdventureWorks has customers but has not yet made any sales by comparing `SalesTerritory` and `SalesOrderHeader` data.


In [19]:
SELECT DISTINCT ST.TerritoryID
FROM Sales.Customer as C
INNER JOIN Sales.SalesTerritory as ST on C.TerritoryID = ST.TerritoryID

EXCEPT

SELECT DISTINCT TerritoryID
FROM Sales.SalesOrderHeader


(0 rows affected)

Total execution time: 00:00:00.004

TerritoryID


## 3. Products Sold vs Products Manufactured:
Find which products have been manufactured but have never been sold by comparing the `Product` and `SalesOrderDetail` tables.


In [27]:
WITH Product_NS AS
(
    (
        SELECT ProductID
        FROM Production.Product
    )
    EXCEPT
    (
        SELECT DISTINCT ProductID
        FROM Sales.SalesOrderDetail
    )
)

SELECT TOP(5) P.Name, P.ProductNumber, P.ListPrice
FROM Product_NS as NS
INNER JOIN Production.Product as P on NS.ProductID = P.ProductID
ORDER BY P.ListPrice DESC

(5 rows affected)

Total execution time: 00:00:00.028

Name,ProductNumber,ListPrice
"HL Road Frame - Black, 58",FR-R92B-58,1431.50
"HL Road Frame - Red, 58",FR-R92R-58,1431.50
"HL Road Frame - Red, 52",FR-R92R-52,1431.50
"HL Road Frame - Red, 56",FR-R92R-56,1431.50
"HL Road Frame - Black, 62",FR-R92B-62,1431.50


## 4. Sales Representatives who are Customers:
Identify the sales representatives who are also listed as customers by intersecting the `SalesPerson` and `Customer` tables.


In [1]:
SELECT BusinessEntityID
FROM Sales.SalesPerson

INTERSECT

SELECT CustomerID
FROM Sales.Customer;


(17 rows affected)

Total execution time: 00:00:00.099

BusinessEntityID
285
283
278
279
277
275
288
287
276
282


## 5. Union of Products in Different Categories:
Generate a combined list of all products in the `Bikes` and `Clothing` categories by using a union of their respective product categories.


In [42]:
DROP FUNCTION IF EXISTS Production.SelectProductByCategory;
GO
CREATE OR ALTER FUNCTION Production.SelectProductByCategory(@Category VARCHAR(10))
RETURNS TABLE
RETURN(
    SELECT  P.Name as [Product Name], P.ProductID, P.ListPrice, PC.Name as [Category Name]
    FROM Production.Product as P
    INNER JOIN Production.ProductSubcategory as PSC ON P.ProductSubcategoryID = PSC.ProductSubcategoryID
    INNER JOIN Production.ProductCategory as PC ON PSC.ProductCategoryID = PC.ProductCategoryID
    WHERE PC.Name = @Category
);
GO

DECLARE @BikesName VARCHAR(10) = 'Bikes';
DECLARE @ClothingName VARCHAR(10) = 'Clothing';

(
    SELECT TOP (5) [Product Name], ProductID, ListPrice, [Category Name]
    FROM Production.SelectProductByCategory(@BikesName)
)

UNION

(
    SELECT TOP(5) [Product Name], ProductID, ListPrice, [Category Name]
    FROM Production.SelectProductByCategory(@ClothingName)
)

Commands completed successfully.

Commands completed successfully.

(10 rows affected)

Total execution time: 00:00:00.066

Product Name,ProductID,ListPrice,Category Name
AWC Logo Cap,712,8.99,Clothing
"Half-Finger Gloves, S",858,24.49,Clothing
"Men's Bib-Shorts, L",857,89.99,Clothing
"Men's Bib-Shorts, M",856,89.99,Clothing
"Men's Bib-Shorts, S",855,89.99,Clothing
"Mountain-100 Black, 38",775,3374.99,Bikes
"Mountain-100 Silver, 38",771,3399.99,Bikes
"Mountain-100 Silver, 42",772,3399.99,Bikes
"Mountain-100 Silver, 44",773,3399.99,Bikes
"Mountain-100 Silver, 48",774,3399.99,Bikes


## 6. Sales Orders without Shipments:
Determine which sales orders have not been shipped by comparing the `SalesOrderHeader` and `ShipMethod` tables.


In [10]:
SELECT SalesOrderID
FROM Sales.SalesOrderHeader
WHERE ShipMethodID IS NULL;

(0 rows affected)

Total execution time: 00:00:00.003

SalesOrderID


## 7. All Employees and All Salespeople:
Create a combined list of all employees and salespeople, ensuring that each individual appears only once by using `UNION`.


In [12]:
SELECT TOP 5 BusinessEntityID
FROM HumanResources.Employee

UNION

SELECT TOP 5 BusinessEntityID
FROM Sales.SalesPerson;


(10 rows affected)

Total execution time: 00:00:00.010

BusinessEntityID
1
2
3
4
5
274
275
276
277
278


## 8. Union of Orders by Country:
Combine all sales orders from customers located in the United States and Canada by querying the `SalesOrderHeader` and `Customer` tables, and then using `UNION`.


In [15]:
SELECT TOP 5 SalesOrderID
FROM Sales.SalesOrderHeader SOH
JOIN Sales.Customer C ON SOH.CustomerID = C.CustomerID
WHERE C.TerritoryID = (SELECT TerritoryID FROM Sales.SalesTerritory WHERE Name = 'United States')

UNION

SELECT TOP 5 SalesOrderID
FROM Sales.SalesOrderHeader SOH
JOIN Sales.Customer C ON SOH.CustomerID = C.CustomerID



(5 rows affected)

Total execution time: 00:00:00.014

SalesOrderID
43697
43698
43699
43700
43701


## 9. Difference in Online vs Physical Store Customers:
Find customers who have placed orders online but not in a physical store by comparing `SalesOrderHeader` and `SalesOrderDetail` with a set difference operation.


In [9]:
SELECT TOP 5 CustomerID
FROM Sales.SalesOrderHeader
WHERE OnlineOrderFlag = 1

EXCEPT

SELECT TOP 5 CustomerID
FROM Sales.SalesOrderHeader
WHERE OnlineOrderFlag = 0;


(5 rows affected)

Total execution time: 00:00:00.013

CustomerID
11003
14501
21768
25863
28389


## 10. Common Customers for Two Products:
Determine which customers have purchased both `Product A` and `Product B` by using an intersection of their respective sales orders.


In [7]:

DROP FUNCTION IF EXISTS Sales.SelectCustomersPurchased;
GO
CREATE OR ALTER FUNCTION Sales.SelectCustomersPurchased(@Product INT)
RETURNS TABLE
RETURN(
    SELECT DISTINCT SOH.CustomerID
    FROM Sales.SalesOrderHeader as SOH
    INNER JOIN Sales.SalesOrderDetail as SOD on SOH.SalesOrderID = SOD.SalesOrderID
    WHERE SOD.ProductID = @Product

);
GO
DECLARE @ProductA  INT = 717;
DECLARE @ProductB INT = 718;

SELECT CustomerID
FROM Sales.SelectCustomersPurchased(@ProductA) as SCP

INTERSECT


SELECT CustomerID
FROM Sales.SelectCustomersPurchased(@ProductB) as SCP


Commands completed successfully.

Commands completed successfully.

(85 rows affected)

Total execution time: 00:00:00.059

CustomerID
29486
29489
29513
29528
29533
29539
29543
29546
29560
29569
